In [1]:
# Use the Cor-Power WEC outline.

# Plot 1:
# Run a dummy Capytaine round to make sure the initial build happens and we're not timing that
# Run Capytaine with evenly-spaced increasing resolution, get times and values.
# Run MEEM with SAME number of terms/region, evenly-spaced increasing resolution, get times and values.
# Designated value at terms = 300 for MEEM.
# All runs should be repeated 25 times and time-averaged to reduce variability.
# Plot the values on the same plot. x = time, y = value.

# Plot 2:
# Stacked plot for a 2-region configuration, how much time each component takes for same terms/region.
# Terms on x, times on y.

# Plot 3:
# Which calculation dominates if you have y regions and x terms/region? Shares legend and colors with plot 2.

In [2]:
import numpy as np
import pickle
from typing import List
import openflash
print(type(openflash))
print(openflash.__path__)
print(openflash.__file__)

# --- Import core modules from package ---
try:
    from openflash import *
    from openflash.multi_equations import *
    from openflash.multi_constants import g
    print("OpenFLASH modules imported successfully!")
except ImportError as e:
    print(f"Error importing OpenFLASH modules. Error: {e}")

# Set NumPy print options for better readability
np.set_printoptions(threshold=np.inf, linewidth=np.inf, precision=8, suppress=True)
from openflash.multi_constants import rho #need for BEM

from time import perf_counter
from capytaine_generator import CapytaineSlantSolver


# Now you can import from the folder structure
# Assuming 'pyplotutilities' is a folder inside 'sea-lab-utils'
import sys
from pathlib import Path
HERE = Path.cwd().resolve()
store_path_prefix = str((HERE / "data").resolve())

<class 'module'>
['/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash']
/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash/__init__.py
OpenFLASH modules imported successfully!


In [3]:
# Variable Definitions
h = 50.00 # sea depth [m]
# omega_sweep = np.linspace(0.4, 1.5, 10) # rad/s
omega_val = 1
d_in = [14.45, 14.45-7.32-5.08] # drafts [m]
d_out = d_in # non-slanted version
a_list = [2.5/2, 8.4/2] # radii [m]
NMK = [100, 100, 100] # number of coefficients in each region
heaving = [1, 1]
face_dist = [a_list[0], d_in[0] - d_in[1], a_list[1] - a_list[0], d_in[1]]
face_fracs = [face_dist[i] / sum(face_dist) for i in range(4)]
cpt_divisions = 15
t_density_lst = [[6 + int(i * 27/cpt_divisions), 12 + int(i * 54/cpt_divisions)] for i in range(cpt_divisions)]
face_unit_lst = [10 + int(i * 40/cpt_divisions) for i in range(cpt_divisions)]
nmk_max = 300
nmk_lst = list(range(5, 101, 5))

config = {"h" : h,
          "a" : a_list,
          "d_in" : d_in,
          "d_out" : d_out,
          "heaving": heaving,
          "omega" : omega_val,
          "rho" : rho}

data_file = store_path_prefix + "/timing.pkl"

In [4]:
css = CapytaineSlantSolver(False, True, False, False, False)
def compute_cpt_slant(config, t_densities, face_units, reps):
  respack = css.construct_and_solve(config["a"], config["d_in"], config["d_out"], config["heaving"], t_densities, face_units, config["h"], config["omega"], config["rho"], reps)
  result, t_diff, result_d, t_diff_d, panel_count = respack
  am, dp = (result.added_mass)["Heave"], (result.radiation_damping)["Heave"]
  return am, dp, t_diff, t_diff_d, panel_count

In [5]:
class TimedMEEMEngine(MEEMEngine):
    def __init__(self, problem_list: List[MEEMProblem]):
        self.timing_dict = {}
        self.timing_dict["Bessels"] = 0
        self.problem_list = problem_list
        self.cache_list = {} 

        for problem in problem_list:
            self.cache_list[problem] = self.build_problem_cache(problem)

    def _ensure_m_k_and_N_k_arrays(self, problem: 'MEEMProblem', m0):
        cache = self.cache_list[problem]
        if cache.m_k_arr is None or cache.cached_m0 != m0:
            domain_list = problem.domain_list
            domain_keys = list(domain_list.keys())
            NMK = [domain_list[idx].number_harmonics for idx in domain_keys]
            h = domain_list[0].h
            t1 = perf_counter()
            m_k_arr = np.array([cache.m_k_entry_func(k, m0, h) for k in range(NMK[-1])])
            t2 = perf_counter()
            N_k_arr = np.array([cache.N_k_func(k, m0, h, m_k_arr) for k in range(NMK[-1])])
            cache._set_precomputed_m_k_N_k(m_k_arr, N_k_arr, m0)
            self.timing_dict["mk"] = t2 - t1

    def build_problem_cache(self, problem: 'MEEMProblem') -> ProblemCache:
        # Redefine the block-creating functions in the engine to the ones that can take
        p_diagonal_block, p_dense_block = self.p_diagonal_block, self.p_dense_block
        v_diagonal_block, v_dense_block = self.v_diagonal_block, self.v_dense_block

        cache = ProblemCache(problem)
        domain_list = problem.domain_list
        domain_keys = list(domain_list.keys())
        
        h = domain_list[0].h
        d = [domain_list[idx].di for idx in domain_keys]
        a = [domain_list[idx].a for idx in domain_keys]
        NMK = [domain.number_harmonics for domain in domain_list.values()]
        heaving = [domain_list[idx].heaving for idx in domain_keys]
        
        boundary_count = len(NMK) - 1
        size = NMK[0] + NMK[-1] + 2 * sum(NMK[1:len(NMK) - 1])

        A_template = np.zeros((size, size), dtype=complex)
        b_template = np.zeros(size, dtype=complex)

        # 2. Pre-compute m0-INDEPENDENT values
        t1 = perf_counter()
        I_nm_vals_precomputed = [np.zeros((NMK[bd], NMK[bd+1]), dtype = complex) for bd in range(boundary_count - 1)]
        for bd in range(boundary_count - 1):
            for n in range(NMK[bd]):
                for m in range(NMK[bd + 1]):
                    I_nm_vals_precomputed[bd][n, m] = I_nm(n, m, bd, d, h)
        t2 = perf_counter()
        self.timing_dict["I_nm"] = t2 - t1
        cache._set_I_nm_vals(I_nm_vals_precomputed)

        R_1n_func = partial(R_1n_vectorized, h=h, d=d, a=a)
        R_2n_func = partial(R_2n_vectorized, a=a, h=h, d=d)
        diff_R_1n_func = partial(diff_R_1n_vectorized, h=h, d=d, a=a)
        diff_R_2n_func = partial(diff_R_2n_vectorized, h=h, d=d, a=a)

        def _calculate_I_mk_vals(m0, m_k_arr, N_k_arr):
            # Optimized to use vectorized ops if possible, but keep loop for now as I_mk is complex
            t1 = perf_counter()
            vals = np.zeros((NMK[boundary_count - 1], NMK[boundary_count]), dtype=complex)
            for m in range(NMK[boundary_count - 1]):
                for k in range(NMK[boundary_count]):
                    vals[m, k] = I_mk(m, k, boundary_count - 1, d, m0, h, m_k_arr, N_k_arr)
            t2 = perf_counter()
            self.timing_dict["I_mk"] = t2 - t1
            return vals

        int_R1_store = {}
        int_R2_store = {}
        int_phi_store = np.zeros(boundary_count, dtype=complex)

        t1 = perf_counter()
        for i in range(boundary_count):
            for n in range(NMK[i]):
                int_R1_store[(i, n)] = int_R_1n(i, n, a, h, d)
        
        for i in range(1, boundary_count):
            for n in range(NMK[i]):
                int_R2_store[(i, n)] = int_R_2n(i, n, a, h, d)
        t2 = perf_counter()
        self.timing_dict["Bessels"] += t2 - t1

        t1 = perf_counter()
        for i in range(boundary_count):
            int_phi_store[i] = int_phi_p_i(i, h, d, a)
        t2 = perf_counter()
        self.timing_dict["Hydros part 1"] = t2 - t1
            
        cache._set_integration_constants(int_R1_store, int_R2_store, int_phi_store)
        cache._set_closure("I_mk_vals", _calculate_I_mk_vals)
        cache._set_m_k_and_N_k_funcs(m_k_entry, N_k_multi)

        ## --- Potential Matching Blocks ---
        col_offset = 0
        row_offset = 0
        for bd in range(boundary_count):
            NMK_inner = NMK[bd]
            NMK_outer = NMK[bd + 1]

            if bd == (boundary_count - 1): 
                # Exterior Boundary (m0-dependent)
                row_height = NMK_inner
                left_block1 = p_diagonal_block(True, R_1n_func, bd, h, d, a, NMK)
                
                if bd > 0:
                    left_block2 = p_diagonal_block(True, R_2n_func, bd, h, d, a, NMK)
                    block = np.concatenate([left_block1, left_block2], axis=1)
                else:
                    block = left_block1
                
                A_template[row_offset : row_offset + row_height, col_offset : col_offset + block.shape[1]] = block
                
                p_dense_e_col_start = col_offset + block.shape[1]
                
                # --- OPTIMIZED POTENTIAL BLOCK ---
                # A[m, k] = -1 * Lambda_k(k) * I_mk(m, k)
                # This broadcasts a row vector (Lambda) against the matrix (I_mk)
                
                # We define a function that will be called in assemble_A_multi
                def potential_block_func(p, m0, mk, Nk, Imk):
                    # Imk is shape (N, M). 
                    # Calculate vector Lambda for all k (0..M-1)
                    k_indices = np.arange(M)
                    # Use vectorized function. a[bd] is the boundary radius.
                    t1 = perf_counter()
                    lambda_vec = Lambda_k_vectorized(k_indices, a[bd], m0, a, mk)
                    t2 = perf_counter()
                    self.timing_dict["Bessels"] += t2 - t1
                    # Broadcast: -1 * Imk * lambda_row
                    # Imk[m,k] * lambda[k]
                    return -1 * Imk * lambda_vec[None, :]

                cache._add_m0_dependent_block(row_offset, p_dense_e_col_start, potential_block_func)
                # ---------------------------------
                
                col_offset += block.shape[1]

            else: 
                # Internal Boundary
                project_on_left = d[bd] > d[bd+1]
                row_height = NMK_inner if project_on_left else NMK_outer
                blocks = []
                
                if project_on_left:
                    blocks.append(p_diagonal_block(True, R_1n_func, bd, h, d, a, NMK))
                    if bd > 0: blocks.append(p_diagonal_block(True, R_2n_func, bd, h, d, a, NMK))
                    blocks.append(p_dense_block(False, R_1n_func, bd, NMK, a, I_nm_vals_precomputed))
                    blocks.append(p_dense_block(False, R_2n_func, bd, NMK, a, I_nm_vals_precomputed))
                else: 
                    blocks.append(p_dense_block(True, R_1n_func, bd, NMK, a, I_nm_vals_precomputed))
                    if bd > 0: blocks.append(p_dense_block(True, R_2n_func, bd, NMK, a, I_nm_vals_precomputed))
                    blocks.append(p_diagonal_block(False, R_1n_func, bd, h, d, a, NMK))
                    blocks.append(p_diagonal_block(False, R_2n_func, bd, h, d, a, NMK))

                full_block = np.concatenate(blocks, axis=1)
                A_template[row_offset : row_offset + row_height, col_offset : col_offset + full_block.shape[1]] = full_block
                col_offset += 2*NMK_inner if bd > 0 else NMK_inner
            
            row_offset += row_height

        ## --- Velocity Matching Blocks ---
        col_offset = 0
        for bd in range(boundary_count):
            N = NMK[bd]
            M = NMK[bd + 1]

            if bd == (boundary_count - 1): # Final i-e boundary
                row_height = M
                v_dense_e_col_start = col_offset
                
                # --- OPTIMIZED VELOCITY BLOCKS ---
                
                # 1. R1 Block (always exists)
                # A[m, k] = - diff_R1(k) * I_mk(k, m)  <-- Note indices: k is interior (N), m is exterior (M)
                # Therefore we need (I_mk)^T
                
                # Precompute geometric derivatives (m0-independent!)
                k_indices_N = np.arange(N)
                t1 = perf_counter()
                diff_r1_vec = diff_R_1n_vectorized(k_indices_N, a[bd], bd, h, d, a)
                t2 = perf_counter()
                self.timing_dict["Bessels"] += t2 - t1
                
                def velocity_block_R1_func(p, m0, mk, Nk, Imk):
                    # Imk is (N, M). Transpose to (M, N) to match [row=m, col=k]
                    # Multiply columns k by diff_r1_vec[k]
                    return -1 * Imk.T * diff_r1_vec[None, :]
                
                cache._add_m0_dependent_block(row_offset, v_dense_e_col_start, velocity_block_R1_func)

                # 2. R2 Block (if bd > 0)
                if bd > 0:
                    r2n_col_start = v_dense_e_col_start + N
                    t1 = perf_counter()
                    diff_r2_vec = diff_R_2n_vectorized(k_indices_N, a[bd], bd, h, d, a)
                    t2 = perf_counter()
                    self.timing_dict["Bessels"] += t2 - t1
                    
                    def velocity_block_R2_func(p, m0, mk, Nk, Imk):
                        return -1 * Imk.T * diff_r2_vec[None, :]

                    cache._add_m0_dependent_block(row_offset, r2n_col_start, velocity_block_R2_func)
                
                # 3. Diagonal Block (Exterior wave maker)
                v_diag_e_col_start = col_offset + (2*N if bd > 0 else N)
                
                def velocity_diag_func(p, m0, mk, Nk, Imk):
                    k_indices_M = np.arange(M)
                    # diff_Lambda_k is m0-dependent
                    t1 = perf_counter()
                    val_vec = diff_Lambda_k_vectorized(k_indices_M, a[bd], m0, a, mk)
                    t2 = perf_counter()
                    self.timing_dict["Bessels"] += t2 - t1
                    return np.diag(h * val_vec)
                
                cache._add_m0_dependent_block(row_offset, v_diag_e_col_start, velocity_diag_func)
                # ---------------------------------
                
                col_offset += (2*N if bd > 0 else N)

            else: # Internal i-i boundaries
                project_on_left = d[bd] <= d[bd+1]
                row_height = N if project_on_left else M
                blocks = []
                
                if project_on_left:
                    blocks.append(v_diagonal_block(True, diff_R_1n_func, bd, h, d, NMK, a))
                    if bd > 0: blocks.append(v_diagonal_block(True, diff_R_2n_func, bd, h, d, NMK, a))
                    blocks.append(v_dense_block(False, diff_R_1n_func, bd, I_nm_vals_precomputed, NMK, a))
                    blocks.append(v_dense_block(False, diff_R_2n_func, bd, I_nm_vals_precomputed, NMK, a))
                else: 
                    blocks.append(v_dense_block(True, diff_R_1n_func, bd, I_nm_vals_precomputed, NMK, a))
                    if bd > 0: blocks.append(v_dense_block(True, diff_R_2n_func, bd, I_nm_vals_precomputed, NMK, a))
                    blocks.append(v_diagonal_block(False, diff_R_1n_func, bd, h, d, NMK, a))
                    blocks.append(v_diagonal_block(False, diff_R_2n_func, bd, h, d, NMK, a))

                full_block = np.concatenate(blocks, axis=1)
                A_template[row_offset : row_offset + row_height, col_offset : col_offset + full_block.shape[1]] = full_block
                col_offset += 2*N if bd > 0 else N
            
            row_offset += row_height

        # Assemble b_template 
        index = 0
        for bd in range(boundary_count):
            if bd == (boundary_count - 1):
                for n in range(NMK[-2]):
                    b_template[index] = b_potential_end_entry(n, bd, heaving, h, d, a)
                    index += 1
            else:
                num_entries = NMK[bd + (d[bd] <= d[bd + 1])]
                for n in range(num_entries):
                    b_template[index] = b_potential_entry(n, bd, d, heaving, h, a)
                    index += 1

        for bd in range(boundary_count):
            if bd == (boundary_count - 1):
                for n_local in range(NMK[-1]):
                    calc_func = lambda p, m0, mk, Nk, n=n_local: \
                        b_velocity_end_entry(n, bd, heaving, a, h, d, m0, NMK, mk, Nk)
                    cache._add_m0_dependent_b_entry(index, calc_func)
                    index += 1
            else:
                num_entries = NMK[bd + (d[bd] > d[bd + 1])]
                for n in range(num_entries):
                    b_template[index] = b_velocity_entry(n, bd, heaving, a, h, d)
                    index += 1
                    
        cache._set_A_template(A_template)
        cache._set_b_template(b_template)
        
        return cache

    def solve_linear_system_multi(self, problem: MEEMProblem, m0) -> np.ndarray:
        cache = self.cache_list[problem]
        self._ensure_m_k_and_N_k_arrays(problem, m0)
        A = self.assemble_A_multi(problem, m0)
        b = self.assemble_b_multi(problem, m0) 
        t1 = perf_counter()
        X = linalg.solve(A, b)
        t2 = perf_counter()
        self.timing_dict["Matrix Solve"] = t2 - t1
        # print(f"Linear system solved. Solution vector X shape: {X.shape}")
        return X

    def p_diagonal_block(self, left, radfunction, bd, h, d, a, NMK):
        region = bd if left else (bd + 1)
        sign = 1 if left else (-1)
        t1 = perf_counter()
        arr = np.diag(radfunction(list(range(NMK[region])), a[bd], region))
        t2 = perf_counter()
        self.timing_dict["Bessels"] += t2 - t1
        return sign * (h - d[region]) * arr

    def p_dense_block(self, left, radfunction, bd, NMK, a, I_nm_vals):
        I_nm_array = I_nm_vals[bd]
        if left: # determine which is region to work in and which is adjacent
            region, adj = bd, bd + 1
            sign = 1
            I_nm_array = np.transpose(I_nm_array)
        else:
            region, adj = bd + 1, bd
            sign = -1
        t1 = perf_counter()
        radial_vector = radfunction(list(range(NMK[region])), a[bd], region)
        t2 = perf_counter()
        self.timing_dict["Bessels"] += t2 - t1
        radial_array = np.outer((np.full((NMK[adj]), 1)), radial_vector)
        return sign * radial_array * I_nm_array

    # arguments: diagonal block on left (T/F), vectorized radial eigenfunction, boundary number
    def v_diagonal_block(self, left, radfunction, bd, h, d, NMK, a):
        region = bd if left else (bd + 1)
        sign = (-1) if left else (1)
        t1 = perf_counter()
        arr = np.diag(radfunction(list(range(NMK[region])), a[bd], region))
        t2 = perf_counter()
        self.timing_dict["Bessels"] += t2 - t1
        return sign * (h - d[region]) * arr

    # arguments: dense block on left (T/F), vectorized radial eigenfunction, boundary number
    def v_dense_block(self, left, radfunction, bd, I_nm_vals, NMK, a):
        I_nm_array = I_nm_vals[bd]
        if left: # determine which is region to work in and which is adjacent
            region, adj = bd, bd + 1
            sign = -1
            I_nm_array = np.transpose(I_nm_array)
        else:
            region, adj = bd + 1, bd
            sign = 1
        t1 = perf_counter()
        radial_vector = radfunction(list(range(NMK[region])), a[bd], region)
        t2 = perf_counter()
        self.timing_dict["Bessels"] += t2 - t1
        radial_array = np.outer((np.full((NMK[adj]), 1)), radial_vector)
        return sign * radial_array * I_nm_array
    
    def timed_hydros(self, problem: MEEMProblem, X, m0):
        t1 = perf_counter()
        results_per_mode = self.compute_hydrodynamic_coefficients(problem, X, m0)
        t2 = perf_counter()
        self.timing_dict["Hydros part 2"] = t2 - t1
        return results_per_mode

In [6]:
def timed_MEEM(config, nmk):
  t1 = perf_counter()
  NMK = [nmk for _ in range(len(config["a"]) + 1)]
  arrangement = ConcentricBodyGroup([SteppedBody(a=np.array(config["a"]),
                                                 d=np.array(config["d_in"]),
                                                 slant_angle= np.array([0, 0]),
                                                 heaving=True)])
  geometry = BasicRegionGeometry(body_arrangement=arrangement, h=config["h"], NMK=NMK)
  problem = MEEMProblem(geometry)
  problem.set_frequencies(np.array([omega_val]))
  engine = TimedMEEMEngine(problem_list=[problem])
  assert engine.cache_list[problem].m_k_arr is None
  m0 = wavenumber(omega_val, config["h"])
  X = engine.solve_linear_system_multi(problem, m0)
  results_per_mode = engine.timed_hydros(problem, X, m0)
  t2 = perf_counter()
  data = engine.timing_dict
  data["Total Time"] = (t2 - t1) # - (data["I_nm"] + data["Hydros part 1"] + data["b part 1"]) # This part was run in excess
  data["Hydro real"] = results_per_mode[0]["real"]
  data["Hydro imag"] = results_per_mode[0]["imag"]
  return data

In [7]:
reps = 25
cpt_data = []
_ = compute_cpt_slant(config, [20, 40], 36, 1) # dummy for compilation
for i in range(cpt_divisions):
  am, dp, t_diff, t_diff_d, panel_count = compute_cpt_slant(config, t_density_lst[i], face_unit_lst[i], reps)
  cpt_data.append({"Added Mass" : am,
                   "Damping" : dp,
                   "Computation Time Radiation" : t_diff,
                   "Computation Time Diffraction" : t_diff_d,
                   "Panel Count" : panel_count})
  
meem_data = []
for nmk in nmk_lst:
  rep_set = [timed_MEEM(config, nmk) for _ in range(reps)]
  output = {key : sum([rep_set[i][key] for i in range(reps)])/reps for key in rep_set[0].keys()}
  output["terms per region"] = nmk
  meem_data.append(output)

high_nmk_data_set = timed_MEEM(config, nmk_max)
true_am, true_dp = high_nmk_data_set["Hydro real"], high_nmk_data_set["Hydro imag"]

Panel Count:  920
Panel Count:  72
Panel Count:  100
Panel Count:  175
Panel Count:  253
Panel Count:  325
Panel Count:  435
Panel Count:  535
Panel Count:  637
Panel Count:  780
Panel Count:  946
Panel Count:  1104
Panel Count:  1235
Panel Count:  1442
Panel Count:  1624
Panel Count:  1860


/var/folders/zz/_5443rfn2v1_n4x4gqlv6jxc0000gr/T/ipykernel_44196/245172648.py:285: LinAlgWarning: Ill-conditioned matrix (rcond=3.84051e-29): result may not be accurate.
  X = linalg.solve(A, b)


In [8]:
named_meem_times = ["Coupling Integrals", "Root Finding", "Matrix Solve",
                    "Bessel Functions", "Hydrodynamic Coefficients"]

def reorganize_MEEM_information(data):
  output = {}
  output["Coupling Integrals"] = data["I_nm"] + data["I_mk"]
  output["Root Finding"] = data["mk"]
  output["Matrix Solve"] = data["Matrix Solve"]
  output["Bessel Functions"] = data["Bessels"]
  output["Hydrodynamic Coefficients"] = data["Hydros part 1"] + data["Hydros part 2"]
  output["Total Time"] = data["Total Time"]
  output["Other/Overhead"] = data["Total Time"] - sum([output[key] for key in named_meem_times])
  output["Added Mass"] = data["Hydro real"]
  output["Damping"] = data["Hydro imag"]
  output["nmk"] = data["terms per region"]
  return output

meem_reorganized_data = [reorganize_MEEM_information(data) for data in meem_data]

In [9]:
# Store data
pickle.dump({"True Added Mass" : true_am,
             "True Damping" : true_dp,
             "Capytaine Data" : cpt_data,
             "MEEM Data" : meem_reorganized_data}, open(data_file, 'wb'))